# PyTorch - homework 2: neural networks

-- Prof. Dorien Herremans

Please run the whole notebook with your code and submit the `.ipynb` file on eDimension that includes your answers [so after you run it]. 

In [1]:
from termcolor import colored

student_number="1004365"
student_name="Lee Jet Xuen"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Lee Jet Xuen, number: 1004365


 ## Question 1 -- XOR neural network [3pts]

a) Train an (at least) 2-layer neural network that can solve the XOR problem. Hint: be sure to check both this week and last week's lab. 

b) Check the predictions resulting from your model in the second code box below.


In [2]:
# load your data
import torch
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
X = torch.Tensor([[0,0],[0,1], [1,0], [1,1]])
Y = torch.Tensor([0,1,1,0]).view(-1,1)

# name your model xor
def xor():
  return torch.nn.Sequential(torch.nn.Linear(2,2), torch.nn.Sigmoid(), torch.nn.Linear(2,1), torch.nn.Sigmoid())

xor = xor()

In [4]:
# define your model loss function, optimizer, etc. 
loss_func = torch.nn.BCELoss()
lr_rate = 3e0
momentum = 0.9
epochs = 5000
optimizer = torch.optim.SGD(xor.parameters(), lr=lr_rate, momentum=momentum)

# train the model
for i in range(epochs):
    loss = loss_func(xor(X), Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [5]:
# test your model using the following functions (make sure the output is printed and saved when you submit this notebook):
# depending on how you defined your network you may need to slightly tweek the below prediction function

test = [[0,0],[0,1],[1,1],[1,0]]

for trial in test: 
  Xtest = torch.Tensor(trial)
  y_hat = xor(Xtest)

  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  print("{0} xor {1} = {2}".format(int(Xtest[0]), int(Xtest[1]), prediction))

0 xor 0 = 0
0 xor 1 = 1
1 xor 1 = 0
1 xor 0 = 1


## Question 2  [2pts]

Imagine a neural network model for a multilabel classification task. 

a) Which loss function should you use?

b) The resulting trained modal has a high variance error. Give 4 possible solutions to improve the model. 


```
[your answer here, no coding required]

* answer A
Binary cross-entropy loss function. Element base decision belonging to 
a certain class should not influence the decision for another class.    

* answer B
  - Regularization
  - Dropout
  - Early Stopping
  - Increase size of training set

```


## Question 3 - Improve hit classification [5pts]

Remember the hit predicton dataset from last week? 

a) Improve the model using a multiplayer perceptron. 

b) Make sure to run your models on the GPU. 

c) Tweek the hyperparameters such as number of nodes or layers, or other. Show two possible configurations and explain which works better and very briefly explain why this may be the case. 




In [6]:
# download dataset
!wget https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv
!wget https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv

--2022-06-27 10:39:19--  https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv
Resolving dorax.s3.ap-south-1.amazonaws.com (dorax.s3.ap-south-1.amazonaws.com)... 52.219.160.150
Connecting to dorax.s3.ap-south-1.amazonaws.com (dorax.s3.ap-south-1.amazonaws.com)|52.219.160.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147372 (144K) [text/csv]
Saving to: ‘herremans_hit_1030training.csv.3’

herremans_hit_1030t 100%[===================>] 143.92K   229KB/s    in 0.6s    

2022-06-27 10:39:21 (229 KB/s) - ‘herremans_hit_1030training.csv.3’ saved [147372/147372]

--2022-06-27 10:39:21--  https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv
Resolving dorax.s3.ap-south-1.amazonaws.com (dorax.s3.ap-south-1.amazonaws.com)... 52.219.160.150
Connecting to dorax.s3.ap-south-1.amazonaws.com (dorax.s3.ap-south-1.amazonaws.com)|52.219.160.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36712 (36K) [text/csv]

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/50.021\ AI/hw5/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/50.021 AI/hw5


In [9]:
# load data

device = "cuda"
train_data = pd.read_csv('/content/herremans_hit_1030training.csv')
test_data = pd.read_csv('/content/herremans_hit_1030test.csv')
print(train_data.head(5))

features = torch.FloatTensor(train_data.loc[:, train_data.columns != 'Topclass1030'].values).to(device)
target = torch.FloatTensor(train_data['Topclass1030']).to(device)

   timesignature  beatdifMedian  beatdifrange  beatdif80perc  beatdifvariance  \
0              4      -0.426354      0.159924      -0.430311        -0.201989   
1              1      -0.425639      0.803453      -0.447282        -0.173888   
2              4      -0.363030     -0.642618      -0.398657        -0.277881   
3              4      -0.513649     -0.599269      -0.535959        -0.278743   
4              4      -0.492764     -0.532997      -0.517050        -0.276868   

   beatdifstdev  T1Median     T1min     T1max  T180perc  ...  T9skewness  \
0      0.029204 -0.054548  0.476469 -0.165555 -0.140664  ...    0.942847   
1      0.127218 -0.068266  1.207067 -0.278461 -0.287069  ...    1.584726   
2     -0.403862 -0.702500 -0.767956 -1.392002 -0.956162  ...   -0.256651   
3     -0.414360 -1.525701  0.100812 -2.278600 -1.840325  ...    0.428083   
4     -0.392195 -1.658309 -0.745546 -2.436105 -1.959553  ...    0.375415   

   T9kurtosis  T10variance  T10stdev  T10skewness  T10ku

In [8]:
# code your model 1

class MLP(nn.Module):
  def __init__(self, input_size, num_classes):
      super(MLP, self).__init__()
      self.fc1 = nn.Linear(input_size, 32)
      self.fc2 = nn.Linear(32, 8)
      self.fc3 = nn.Linear(8, num_classes)
      self.relu = nn.ReLU()
      self.sigmoid = nn.Sigmoid()
        
  def forward(self, x):
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      x = self.relu(x)
      x = self.fc3(x)
      x = self.relu(x)
      x = self.sigmoid(x)
      return x

In [10]:
def train(features, target, model, loss_function, optimizer, device, epochs=1000+1):
    for epoch in range(epochs):
        features = features.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        prediction = model(features)
        loss = loss_function(prediction, target.view(-1, 1)).to(device)
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            print (f"Epoch: {epoch}, Loss: {loss}")

In [11]:
def save(model, filepath):
  torch.save(model.state_dict(), filepath)
  print(f"model saved in {filepath}")

In [12]:
dim_out = 1
dim_inp = 49 
lr_rate = 0.001
loss_function = nn.BCELoss()

model1 = MLP(dim_inp, dim_out).to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=lr_rate)

train(features, target, model1, loss_function, optimizer, device)

Epoch: 0, Loss: 0.684018611907959
Epoch: 50, Loss: 0.5774528980255127
Epoch: 100, Loss: 0.4865177869796753
Epoch: 150, Loss: 0.3989968001842499
Epoch: 200, Loss: 0.32910582423210144
Epoch: 250, Loss: 0.2921326160430908
Epoch: 300, Loss: 0.2758706212043762
Epoch: 350, Loss: 0.2705480456352234
Epoch: 400, Loss: 0.26831066608428955
Epoch: 450, Loss: 0.26691052317619324
Epoch: 500, Loss: 0.26623454689979553
Epoch: 550, Loss: 0.26555001735687256
Epoch: 600, Loss: 0.2651634216308594
Epoch: 650, Loss: 0.2649206221103668
Epoch: 700, Loss: 0.2646608352661133
Epoch: 750, Loss: 0.26456940174102783
Epoch: 800, Loss: 0.2643705904483795
Epoch: 850, Loss: 0.2642989158630371
Epoch: 900, Loss: 0.2641901671886444
Epoch: 950, Loss: 0.2640797197818756
Epoch: 1000, Loss: 0.26403507590293884


In [13]:
# model 2

class DropoutMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DropoutMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, num_classes)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.75)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.sigmoid(x)
        return x
    
# train model 2

model2 = DropoutMLP(dim_inp, dim_out).to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=lr_rate)

train(features, target, model2, loss_function, optimizer, device)

Epoch: 0, Loss: 0.6931471824645996
Epoch: 50, Loss: 0.6931471824645996
Epoch: 100, Loss: 0.6931471824645996
Epoch: 150, Loss: 0.6931471824645996
Epoch: 200, Loss: 0.6931471824645996
Epoch: 250, Loss: 0.6931471824645996
Epoch: 300, Loss: 0.6541837453842163
Epoch: 350, Loss: 0.5725418329238892
Epoch: 400, Loss: 0.5523510575294495
Epoch: 450, Loss: 0.5257366895675659
Epoch: 500, Loss: 0.49477481842041016
Epoch: 550, Loss: 0.47855183482170105
Epoch: 600, Loss: 0.45215079188346863
Epoch: 650, Loss: 0.4181925356388092
Epoch: 700, Loss: 0.44576355814933777
Epoch: 750, Loss: 0.4200247824192047
Epoch: 800, Loss: 0.4148452877998352
Epoch: 850, Loss: 0.3968169093132019
Epoch: 900, Loss: 0.3924323320388794
Epoch: 950, Loss: 0.3819860517978668
Epoch: 1000, Loss: 0.36812108755111694


In [14]:
save(model1, "model1")
save(model2, "model2")

model saved in model1
model saved in model2


In [15]:
# evaluate model 1 (called model1 here)


import pandas as pd 

def run_evaluation(my_model):

  test = pd.read_csv('/content/herremans_hit_1030test.csv')
  labels = test.iloc[:,-1]
  test = test.drop('Topclass1030', axis=1)
  testdata = torch.Tensor(test.values)
  testlabels = torch.Tensor(labels.values).view(-1,1)

  TP = 0
  TN = 0
  FN = 0
  FP = 0

  for i in range(0, testdata.size()[0]): 
    # print(testdata[i].size())
    Xtest = torch.Tensor(testdata[i]).to(device)
    y_hat = my_model(Xtest)
    
    if y_hat > 0.5:
      prediction = 1
    else: 
      prediction = 0

    if (prediction == testlabels[i]):
      if (prediction == 1):
        TP += 1
      else: 
        TN += 1

    else:
      if (prediction == 1):
        FP += 1
      else: 
        FN += 1

  print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
  print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
  rate = TP/(FN+TP)
  print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

#### to read the model file

In [16]:
# device = "cuda"
# model1 = MLP(dim_inp, dim_out).to(device)
# model1.load_state_dict(torch.load("model1"))

# model2 = DropoutMLP(dim_inp, dim_out).to(device)
# model2.load_state_dict(torch.load("model2"))

In [17]:
# evaluate on model 1
model1.eval()
run_evaluation(model1)

True Positives: 45, True Negatives: 10
False Positives: 19, False Negatives: 5
Class specific accuracy of correctly predicting a hit song is 0.9


In [18]:
# evaluate model 2 (called model2 here)
model2.eval()
run_evaluation(model2)

True Positives: 47, True Negatives: 10
False Positives: 19, False Negatives: 3
Class specific accuracy of correctly predicting a hit song is 0.94


Which works better and why do you think this may be (very briefly)? 


**[your answer here, also please summarise the differences between your two models]**

Both models are 3 layers logistic regression classifier, with the same optimizer and loss function. However, `model2` has a **dropout layer** after the first fully connected layer

The model 2 performs better than model 1 in terms of class specific accuracy, and predicted more true positives than model 1. The dropout layer force the model to not rely on any single particular node when training as it will be turned off randomly. This helps in prevent overfitting.

Additionally, submit your results [here](https://forms.gle/NtJJEE7Wm5ZRM3Je7) for 'Class specific accuracy of correctly predicting a hit song' and see if you got the best performance of the class! Good luck!